
# Analysis Notebook

This notebook can be used to calculate statistical metrics for the data produced. 
*** 

## 0. Setup <a class="anchor" id="0"></a>
This section imports all files and sets up the notebook.

In [169]:
# Import all pacakges
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [170]:
# Choose which results to use
results_to_use = [""]  # If this is empty, all files in the folder will be used
WARMUP_EPOCHS = 0 # Number of epochs to ignore.
sim_colours = ['blue', 'green', 'red', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']


all_results = os.listdir('results/')

if results_to_use == [""]:
    results_to_use = all_results
# remove DS_Store from the list
if '.DS_Store' in results_to_use:
    results_to_use.remove('.DS_Store')
    
print("Using results: " + str(results_to_use))


Using results: ['random', 'adaptive']


In [171]:
# Create a dictionary. Each key is a set of results. 

all_sim_results = {}

for sim in results_to_use:
    all_sim_results[sim] = {}
    all_sim_results[sim]['configuration'] = pd.read_csv(
        'results/' + sim + '/configuration.txt', header=None, delimiter=":")
    types_of_results = []
    # Get all the filenames in the folder, excluding the extension
    for file in os.listdir('results/' + sim + '/exported_data/'):
        if file.endswith(".npy"):
            types_of_results.append(file[:-4])
    # Create a dictionary for each type of result
    for result in types_of_results:
        all_sim_results[sim][result] = np.load(
            'results/' + sim + '/exported_data/' + result + '.npy')

In [172]:
# Show all the all_sim_results in the notebook
for set_of_results in results_to_use:
    print("Results: " + set_of_results)
    print(all_sim_results[set_of_results]['configuration'])

Results: random
                                    0                   1
0                             command                 run
1                       num_of_epochs                1000
2                  num_of_simulations                  10
3                           grid_size                   3
4                      queue_capacity                  10
5                     congestion_rate                0.07
6                       with_hotspots                True
7                           wage_time                  10
8                      credit_balance                   5
9           shared_auction_parameters                None
10  adaptive_auction_action_selection              random
11               bid_calculation_rule              linear
12       adaptive_auction_update_rule       simple_bandit
13             auction_episode_length                  10
14   action_selection_hyperparameters                None
15    adaptive_auction_discretization                  2

## 1. Average Total Number of Trips  <a class="anchor" id="1"></a>

This t-test compares the average total number of trips between the two experiments, for all sims.

In [173]:
all_average_number_of_trips = []
for sim in results_to_use:
    all_average_number_of_trips.append(
        all_sim_results[sim]['stat_num_of_trips_per_simulation'])

results = stats.ttest_ind(all_average_number_of_trips[0], all_average_number_of_trips[1])
print(results)
#present the results nicely
print("The average number of trips for the first two simulations are: " + str(np.mean(all_average_number_of_trips[0])) + " and " + str(np.mean(all_average_number_of_trips[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=-0.5452943009439454, pvalue=0.5922446263812466, df=18.0)
The average number of trips for the first two simulations are: 2485.3 and 2495.5
The p-value is: 0.5922446263812466
The difference is not significant


## 2. Average Congestion of central Intersection

This t-test is between the 2 experiments, for all sims. It does not process individual epochs

In [174]:
average_congestion_per_intersection = []
for exp in results_to_use:
    average_congestion_per_intersection.append(
        all_sim_results[exp]['stat_average_congestion_per_intersection'])

results = stats.ttest_ind(average_congestion_per_intersection[0][:,1,1], average_congestion_per_intersection[1][:,1,1])
print(results)
print("The average congestion for the first two simulations are: " + str(np.mean(average_congestion_per_intersection[0][:,1,1])) + " and " + str(np.mean(average_congestion_per_intersection[1][:,1,1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")

TtestResult(statistic=1.6913295535850714, pvalue=0.10801435313223114, df=18.0)
The average congestion for the first two simulations are: 0.33702 and 0.3249
The p-value is: 0.10801435313223114
The difference is not significant


## 3. Average Time Waited <a id="3"></a>

agent, intersection and grid based

In [175]:
######Agent based######
# Average time
average_time_agent = []
for exp in results_to_use:
    average_time_agent.append(
        all_sim_results[exp]['stat_average_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(average_time_agent[0], average_time_agent[1])
print(results)
print("The average time waited for the first two simulations are: " + str(np.mean(average_time_agent[0])) + " and " + str(np.mean(average_time_agent[1])))
print("The p-value is: " + str(results[1]))

# Max time
max_time_agent = []
for exp in results_to_use:
    max_time_agent.append(
        all_sim_results[exp]['stat_max_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(max_time_agent[0], max_time_agent[1])
print(results)
print("The max time waited for the first two simulations are: " + str(np.mean(max_time_agent[0])) + " and " + str(np.mean(max_time_agent[1])))
print("The p-value is: " + str(results[1]))

TtestResult(statistic=2.023249528344651, pvalue=0.0581565315375937, df=18.0)
The average time waited for the first two simulations are: 0.26753611111111114 and 0.2560444444444444
The p-value is: 0.0581565315375937
TtestResult(statistic=2.2452696182175007, pvalue=0.03755015527388474, df=18.0)
The max time waited for the first two simulations are: 0.6867444444444446 and 0.6574444444444444
The p-value is: 0.03755015527388474


In [176]:
######Intersection based - Average Time######

average_time_intersection = []
for exp in results_to_use:
    average_time_intersection.append(
        all_sim_results[exp]['stat_average_time_waited_per_intersection'])
    
for i in range(len(average_time_intersection[0][0])):
    for j in range(len(average_time_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_time_intersection[0][:,i,j], average_time_intersection[1][:,i,j])
        print(results)
        print("The average time waited for the first two simulations are: " + str(np.mean(average_time_intersection[0][:,i,j])) + " and " + str(np.mean(average_time_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=-1.8970504612320969, pvalue=0.07398422302534072, df=18.0)
The average time waited for the first two simulations are: 0.019625 and 0.027174999999999998
The p-value is: 0.07398422302534072
Intersection [ 0 ][ 1 ]
TtestResult(statistic=-2.4082145313917778, pvalue=0.026968070487147056, df=18.0)
The average time waited for the first two simulations are: 0.149675 and 0.20692500000000003
The p-value is: 0.026968070487147056
Intersection [ 0 ][ 2 ]
TtestResult(statistic=-0.3136148531511767, pvalue=0.7574187859159365, df=18.0)
The average time waited for the first two simulations are: 0.0428 and 0.047700000000000006
The p-value is: 0.7574187859159365
Intersection [ 1 ][ 0 ]
TtestResult(statistic=0.46855223284145026, pvalue=0.6450140707051305, df=18.0)
The average time waited for the first two simulations are: 0.32175000000000004 and 0.30199999999999994
The p-value is: 0.6450140707051305
Intersection [ 1 ][ 1 ]
TtestResult(statistic=6.000813573756642

In [177]:
######Intersection based - Max Time######

max_time_intersection = []
for exp in results_to_use:
    max_time_intersection.append(
        all_sim_results[exp]['stat_max_time_waited_per_intersection'])
    
for i in range(len(max_time_intersection[0][0])):
    for j in range(len(max_time_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(max_time_intersection[0][:,i,j], max_time_intersection[1][:,i,j])
        print(results)
        print("The average time waited for the first two simulations are: " + str(np.mean(max_time_intersection[0][:,i,j])) + " and " + str(np.mean(max_time_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=-1.9769712392283159, pvalue=0.06356872643327098, df=18.0)
The average time waited for the first two simulations are: 0.0715 and 0.0974
The p-value is: 0.06356872643327098
Intersection [ 0 ][ 1 ]
TtestResult(statistic=-2.572104159826212, pvalue=0.019190888219053712, df=18.0)
The average time waited for the first two simulations are: 0.43699999999999994 and 0.5892000000000001
The p-value is: 0.019190888219053712
Intersection [ 0 ][ 2 ]
TtestResult(statistic=-0.25954120432455186, pvalue=0.7981629190290955, df=18.0)
The average time waited for the first two simulations are: 0.1354 and 0.14500000000000002
The p-value is: 0.7981629190290955
Intersection [ 1 ][ 0 ]
TtestResult(statistic=0.2516215210495754, pvalue=0.8041839743356436, df=18.0)
The average time waited for the first two simulations are: 0.8515 and 0.8262
The p-value is: 0.8041839743356436
Intersection [ 1 ][ 1 ]
TtestResult(statistic=6.488016163444629, pvalue=4.21358731016062e-06, df=

In [178]:
######Grid based######

# Average time
print("AVERAGE TIME WAITED")
average_time_grid = []
for exp in results_to_use:
    average_time_grid.append(
        all_sim_results[exp]['stat_average_time_waited_grid'])

means_1 = []
for i in range(len(average_time_grid[0])):
    means_1.append(np.mean(average_time_grid[0][i]))
means_2 = []
for i in range(len(average_time_grid[1])):
    means_2.append(np.mean(average_time_grid[1][i]))


results = stats.ttest_ind(means_1, means_2)
print(results)
print("The average time waited for the first two simulations are: " + str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
print("The p-value is: " + str(results[1]))


# Max time
print("MAX TIME WAITED")
max_time_grid = []
for exp in results_to_use:
    max_time_grid.append(
        all_sim_results[exp]['stat_max_time_waited_grid'])

means_1 = []
for i in range(len(max_time_grid[0])):
    means_1.append(np.mean(max_time_grid[0][i]))
means_2 = []
for i in range(len(max_time_grid[1])):
    means_2.append(np.mean(max_time_grid[1][i]))


results = stats.ttest_ind(means_1, means_2)
print(results)
print("The max time waited for the first two simulations are: " + str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
print("The p-value is: " + str(results[1]))

AVERAGE TIME WAITED
TtestResult(statistic=2.0232495283446528, pvalue=0.05815653153759351, df=18.0)
The average time waited for the first two simulations are: 0.26753611111111114 and 0.2560444444444444
The p-value is: 0.05815653153759351
MAX TIME WAITED
TtestResult(statistic=2.2452696182175003, pvalue=0.03755015527388476, df=18.0)
The max time waited for the first two simulations are: 0.6867444444444446 and 0.6574444444444444
The p-value is: 0.03755015527388476


## 4. Gini coefficient <a id="4"></a>

In [179]:
# Average Time Waited.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_time_waited = []
for sim in results_to_use:
    all_ginis_time_waited.append(
        all_sim_results[sim]['stat_time_waited_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
#present the results nicely
print("The average gini of time waited for the first two simulations are: " + str(np.mean(all_ginis_time_waited[0])) + " and " + str(np.mean(all_ginis_time_waited[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=-1.971440403280763, pvalue=0.06424488138991127, df=18.0)
The average gini of time waited for the first two simulations are: 0.13040286664715445 and 0.13625529917560644
The p-value is: 0.06424488138991127
The difference is not significant


In [180]:
# Average Satisfaction.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_satisfaction = []
for sim in results_to_use:
    all_ginis_satisfaction.append(
        all_sim_results[sim]['stat_satisfaction_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
#present the results nicely
print("The average gini of satisfaction for the first two simulations are: " + str(np.mean(all_ginis_satisfaction[0])) + " and " + str(np.mean(all_ginis_satisfaction[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=-1.971440403280763, pvalue=0.06424488138991127, df=18.0)
The average gini of satisfaction for the first two simulations are: 0.43637502559342795 and 0.43060879686487585
The p-value is: 0.06424488138991127
The difference is not significant


## 5. Trip Satisfaction <a id="5"></a>

In [181]:
all_satisfactions = []
for sim in results_to_use:
    all_satisfactions.append(
        all_sim_results[sim]['stat_satisfaction_mean'])

results = stats.ttest_ind(all_satisfactions[0], all_satisfactions[1])
print(results)
#present the results nicely
print("The average satisfactino for the first two simulations are: " + str(np.mean(all_satisfactions[0])) + " and " + str(np.mean(all_satisfactions[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=-0.5380463119818749, pvalue=0.5971358593061851, df=18.0)
The average satisfactino for the first two simulations are: 0.15539593185876988 and 0.15611872149685274
The p-value is: 0.5971358593061851
The difference is not significant
